In [1]:
from numpy import random
from tqdm import tqdm

In [2]:
# 随机分数，正态分布
def random_score(low, high):
    rand_val = random.normal(0.5,0.15)
    score = int(max(0, min(1, rand_val)) * (high-low) + low)
    return score

In [3]:
# 随机学校数，均匀分布
def random_school_num(low, high):
    return random.randint(low, high)

In [4]:
# 随机每个学校的学生数，均匀分布
def random_student_num(low, high):
    return random.randint(low, high)

In [5]:
# 生成每个学校的总分列表
def school_score_list(student_num):
    scores = [random_score(LOW_SCO, HIG_SCO) for x in range(0, student_num)]
    return scores

In [6]:
# 根据学校平均分，以及人数分配指标，平均分越高指标越多，人数越多指标越多，但具有一定随机性
def assign_enroll_num(school_num):
    avg_score_list = []
    score_matrix = []
    stu_num_list = []
    # 统计分数，学生数
    for idx in range(0, school_num):
        stu_num = random_student_num(MIN_STU, MAX_STU)
        stu_num_list.append(stu_num)
        scores = school_score_list(stu_num)
        score_matrix.append(scores)
        avg_score = sum(scores)/len(scores)
        avg_score_list.append(avg_score)
        
    total_score = sum(avg_score_list)
    total_stu_num = sum(stu_num_list)
    score_ratio_list = [x/total_score for x in avg_score_list]
    stu_ratio_list = [x/total_stu_num for x in stu_num_list]
    noised_ratio_list = []
    # 计算分配指标占比，并添加噪音
    for idx, stu_ratio in enumerate(stu_ratio_list):
        score_ratio = score_ratio_list[idx]
        stu_num = stu_num_list[idx]
        noise = (random.random() - 0.5) * NOS_LEV * stu_num
        noised_ratio_list.append(noise + score_ratio + stu_ratio)
    # 归一化
    sum_ratio = sum(noised_ratio_list)
    normed_ratio_list = [x/sum_ratio for x in noised_ratio_list]
    # 计算招生数
    total_enroll_num = int(total_stu_num * random.uniform(MIN_RAT, MAX_RAT)) # 总共录取人数
    before_num = int(random.uniform(MIN_BEF, MAX_BEF) * total_enroll_num) # 提前批录取人数
    enroll_num_list = [int(x * before_num) for x in normed_ratio_list] # 每个学校提前批录取人数
    for idx, stu_num in enumerate(stu_num_list):
        enroll_num = enroll_num_list[idx]
        if enroll_num - stu_num > 0:
            return False, False, False, False
    return total_enroll_num, before_num, enroll_num_list, score_matrix

In [7]:
# 情况1：先每个学生按指标招统招，然后从剩下的里面选分数最高的
def case_one(school_num, total_enroll_num, before_num, enroll_num_list, score_matrix):
    next_scores = [] # 第二批候选池
    next_num = total_enroll_num - before_num
    before_score_matrix = []
    for i in range(0, school_num):
        scores = score_matrix[i]
        sorted_scores = sorted(scores, reverse=True)
        bef_sch_num = enroll_num_list[i]
        bef_scores = sorted_scores[0:bef_sch_num]
        next_scores = sorted_scores[bef_sch_num:]
        before_score_matrix.append(bef_scores)
        next_scores += next_scores
    sorted_next_scores = sorted(next_scores, reverse=True)
    enrolled_next_scores = sorted_next_scores[:next_num]
    return before_score_matrix, enrolled_next_scores

In [8]:
# 情况2：先选最高的，再从剩下的选统招
def case_two(school_num, total_enroll_num, before_num, enroll_num_list, score_matrix):
    total_score_list = []
    next_num = total_enroll_num - before_num
    for score in score_matrix:
        total_score_list += score
    sorted_total_score_list = sorted(total_score_list, reverse=True)
    next_scores = sorted_total_score_list[:next_num]
    before_score_matrix = []
    
    # 排除选走的非统招
    for next_score in next_scores:
        shuf_idx = [x for x in range(0, school_num)]
        random.shuffle(shuf_idx)
        
        for idx in shuf_idx:
            if next_score in score_matrix[idx]:
                score_matrix[idx].remove(next_score)
                break
                
    # 从剩下的选统招
    for i in range(0, school_num):
        scores = score_matrix[i]
        sorted_scores = sorted(scores, reverse=True)
        bef_sch_num = enroll_num_list[i]
        bef_scores = sorted_scores[0:bef_sch_num]
        before_score_matrix.append(bef_scores)
        
    return before_score_matrix, next_scores



In [9]:
def cal_reward(before_score_matrix, next_scores):
    before_score_matrix.append(next_scores)
    total = 0
    for score_list in before_score_matrix:
        total += sum(score_list)
    return total

In [10]:
def simulate(sim_num = 10):
    case_one_win = 0
    case_two_win = 0
    draw_num = 0
    for i in tqdm(range(sim_num)):
        school_num = random_school_num(MIN_SCH, MAX_SCH)
        
        total_enroll_num, before_num, enroll_num_list, score_matrix = assign_enroll_num(school_num)
        while total_enroll_num == False:
            total_enroll_num, before_num, enroll_num_list, score_matrix = assign_enroll_num(school_num)
        before_score_matrix1, next_scores1 = case_one(school_num, total_enroll_num, before_num, enroll_num_list, score_matrix)
        before_score_matrix2, next_scores2 = case_two(school_num, total_enroll_num, before_num, enroll_num_list, score_matrix)
        case_one_reward = cal_reward(before_score_matrix1, next_scores1)
        case_two_reward = cal_reward(before_score_matrix2, next_scores2)
        
        # print (next_scores2)
        if case_one_reward > case_two_reward:
            case_one_win += 1
        elif case_one_reward < case_two_reward:
            case_two_win += 1
        else:
            # print (case_one_reward, case_two_reward)
            draw_num += 1
        # print (case_one_reward, case_two_reward)
        
    print('case_one_win: ', case_one_win)
    print('case_two_win: ', case_two_win)
    print('draw_num: ', draw_num)


In [11]:
# baseline
LOW_SCO = 0  # 最低分
HIG_SCO = 500 # 最高分
MIN_SCH = 5 # 最少学校数
MAX_SCH = 10 # 最多学校数
MIN_STU = 500 # 每个学校最少学生数
MAX_STU = 1000 # 每个学校最多学生数
NOS_LEV = 0.0001 # 模拟招生指标事前分配，取决于学生数和教学水平，随机性越大值越高，0是事后分配
MIN_RAT = 0.1 # 最低总录取率
MAX_RAT = 0.2 # 最高总录取率
MIN_BEF = 0.2 # 最低统招占比
MAX_BEF = 0.8 # 最高统招占比
simulate(sim_num = 10000)

100%|██████████| 10000/10000 [04:22<00:00, 38.03it/s]

case_one_win:  40
case_two_win:  9960
draw_num:  0


In [12]:
# 提升区域学校数
LOW_SCO = 0  # 最低分
HIG_SCO = 500 # 最高分
MIN_SCH = 20 # 最少学校数
MAX_SCH = 40 # 最多学校数
MIN_STU = 500 # 每个学校最少学生数
MAX_STU = 1000 # 每个学校最多学生数
NOS_LEV = 0.0001 # 模拟招生指标事前分配，取决于学生数和教学水平，随机性越大值越高，0是事后分配
MIN_RAT = 0.1 # 最低总录取率
MAX_RAT = 0.2 # 最高总录取率
MIN_BEF = 0.2 # 最低统招占比
MAX_BEF = 0.8 # 最高统招占比
simulate(sim_num = 10000)

100%|██████████| 10000/10000 [23:55<00:00,  6.97it/s]

case_one_win:  0
case_two_win:  10000
draw_num:  0


In [13]:
# 提升分配的不公平性
LOW_SCO = 0  # 最低分
HIG_SCO = 500 # 最高分
MIN_SCH = 5 # 最少学校数
MAX_SCH = 10 # 最多学校数
MIN_STU = 500 # 每个学校最少学生数
MAX_STU = 1000 # 每个学校最多学生数
NOS_LEV = 0.001 # 模拟招生指标事前分配，取决于学生数和教学水平，随机性越大值越高，0是事后分配
MIN_RAT = 0.1 # 最低总录取率
MAX_RAT = 0.2 # 最高总录取率
MIN_BEF = 0.2 # 最低统招占比
MAX_BEF = 0.8 # 最高统招占比
simulate(sim_num = 10000)

100%|██████████| 10000/10000 [04:19<00:00, 38.59it/s]

case_one_win:  3584
case_two_win:  6415
draw_num:  1


In [14]:
# 提升区域学校规模
LOW_SCO = 0  # 最低分
HIG_SCO = 500 # 最高分
MIN_SCH = 5 # 最少学校数
MAX_SCH = 10 # 最多学校数
MIN_STU = 5000 # 每个学校最少学生数
MAX_STU = 10000 # 每个学校最多学生数
NOS_LEV = 0.0001 # 模拟招生指标事前分配，取决于学生数和教学水平，随机性越大值越高，0是事后分配
MIN_RAT = 0.1 # 最低总录取率
MAX_RAT = 0.2 # 最高总录取率
MIN_BEF = 0.2 # 最低统招占比
MAX_BEF = 0.8 # 最高统招占比
simulate(sim_num = 10000)

100%|██████████| 10000/10000 [9:15:14<00:00,  3.33s/it]    

case_one_win:  3682
case_two_win:  6318
draw_num:  0


In [15]:
# 降低招生率
LOW_SCO = 0  # 最低分
HIG_SCO = 500 # 最高分
MIN_SCH = 5 # 最少学校数
MAX_SCH = 10 # 最多学校数
MIN_STU = 500 # 每个学校最少学生数
MAX_STU = 1000 # 每个学校最多学生数
NOS_LEV = 0.0001 # 模拟招生指标事前分配，取决于学生数和教学水平，随机性越大值越高，0是事后分配
MIN_RAT = 0.01 # 最低总录取率
MAX_RAT = 0.02 # 最高总录取率
MIN_BEF = 0.2 # 最低统招占比
MAX_BEF = 0.8 # 最高统招占比
simulate(sim_num = 10000)

100%|██████████| 10000/10000 [03:13<00:00, 51.59it/s]

case_one_win:  369
case_two_win:  9626
draw_num:  5


In [16]:
# 提升招生率
LOW_SCO = 0  # 最低分
HIG_SCO = 500 # 最高分
MIN_SCH = 5 # 最少学校数
MAX_SCH = 10 # 最多学校数
MIN_STU = 500 # 每个学校最少学生数
MAX_STU = 1000 # 每个学校最多学生数
NOS_LEV = 0.0001 # 模拟招生指标事前分配，取决于学生数和教学水平，随机性越大值越高，0是事后分配
MIN_RAT = 0.8 # 最低总录取率
MAX_RAT = 0.9 # 最高总录取率
MIN_BEF = 0.2 # 最低统招占比
MAX_BEF = 0.8 # 最高统招占比
simulate(sim_num = 10000)

100%|██████████| 10000/10000 [08:11<00:00, 20.35it/s]

case_one_win:  9
case_two_win:  9991
draw_num:  0


In [17]:
# 降低统招占比
LOW_SCO = 0  # 最低分
HIG_SCO = 500 # 最高分
MIN_SCH = 5 # 最少学校数
MAX_SCH = 10 # 最多学校数
MIN_STU = 500 # 每个学校最少学生数
MAX_STU = 1000 # 每个学校最多学生数
NOS_LEV = 0.0001 # 模拟招生指标事前分配，取决于学生数和教学水平，随机性越大值越高，0是事后分配
MIN_RAT = 0.1 # 最低总录取率
MAX_RAT = 0.2 # 最高总录取率
MIN_BEF = 0.2 # 最低统招占比
MAX_BEF = 0.3 # 最高统招占比
simulate(sim_num = 10000)

100%|██████████| 10000/10000 [04:48<00:00, 34.63it/s]

case_one_win:  0
case_two_win:  10000
draw_num:  0


In [18]:
# 提升统招占比
LOW_SCO = 0  # 最低分
HIG_SCO = 500 # 最高分
MIN_SCH = 5 # 最少学校数
MAX_SCH = 10 # 最多学校数
MIN_STU = 500 # 每个学校最少学生数
MAX_STU = 1000 # 每个学校最多学生数
NOS_LEV = 0.0001 # 模拟招生指标事前分配，取决于学生数和教学水平，随机性越大值越高，0是事后分配
MIN_RAT = 0.1 # 最低总录取率
MAX_RAT = 0.2 # 最高总录取率
MIN_BEF = 0.8 # 最低统招占比
MAX_BEF = 0.9 # 最高统招占比
simulate(sim_num = 10000)

100%|██████████| 10000/10000 [03:29<00:00, 47.75it/s]

case_one_win:  984
case_two_win:  9005
draw_num:  11
